#### [Naver 영화 리뷰 데이터 셋 전처리하기] <hr>

- train; ratings_train.txt
- test: https://github.com/e9t.nsmc/blob/masters/ratings_test.txt

[1]. 데이터 준비 <hr>

In [1]:
#!pip install html5lib

In [2]:
# 데이터 다운로드
from urllib.request import urlretrieve

filename = '../datas/ratings_train.txt'

train_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt"
urlretrieve(url = train_url, filename = filename)


('../datas/ratings_train.txt', <http.client.HTTPMessage at 0x15a821e3580>)

In [3]:
import pandas as pd
data= pd.read_table(filename, usecols=(1,2))
data

,document,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


[2] 데이터 전처리하기 <hr>
* 결측치 / 중복데이터
* 클래스 균형데이터 여부 체크

* 텍스트 데이터 정제(불용어 제거, 노이즈 데이터 제거)
* 토큰화하기
* 단어사전 만들기
* 텍스트 데이터 인코딩
* 텍스트 데이터 패딩

정제는 수시로 발생할 수 있음

In [4]:
data.isnull().sum() # 결측치 확인: document에서 5개 발생. ->id, label은 원래 고유하게 할당되는 녀석이니 의미 없음
data.dropna(inplace = True) #그럼 nropna()해서 제거해버리장.

data.isnull().sum() # 결측치 제거 확인: 없어졋다. 굿~

data[data.duplicated()]
data

,document,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [5]:
## 균형데이터인지 비율 체크하기
data['label'].value_counts() #긍정: 74825, 부정: 75170

label
1    25171
0    24826
Name: count, dtype: int64

In [6]:
#feature / label 분리하기
print(f'columns: {data.columns}')

documentDF = data['document'].to_frame()
labelSR = data['label']

print(f'document: {documentDF.shape}\n\n label: {labelSR}')

columns: Index(['document', 'label'], dtype='object')
document: (49997, 1)

 label: 0        1
1        0
2        0
3        0
4        0
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: label, Length: 49997, dtype: int64


2 - 2. 정제(노이즈데이터, 불용어 제거하기), 토큰화, 정수화하기

노이즈데이터 기준 : 한글만, 한글 + 영어, 한글+영어+숫자

In [7]:
import string

punctuation = string.punctuation
len(punctuation)
hanguel_pattern = "[^ㄱ-ㅎㅏ-ㅣ가-힣]"

In [23]:
# 한글과 공백을 제외하고 모두 제거
import numpy as np
#regex=True : 정규표현식 적용
documentDF['document'] = documentDF['document'].str.replace(hanguel_pattern, '', regex=True)
#documentDF['document'].replace('',value=np.nan,inplace=True)
documentDF['document'][:5]

0                               굳ㅋ
1                              NaN
2         뭐야이평점들은나쁘진않지만점짜리는더더욱아니잖아
3             지루하지는않은데완전막장임돈주고보기에는
4    만아니었어도별다섯개줬을텐데왜로나와서제심기를불편하게하죠
Name: document, dtype: object

In [38]:
import numpy as np
documentDF['document'] = documentDF['document'].str.replace('^ +', "", regex=True)
documentDF['document'].replace('',value=np.nan,inplace=True)

documentDF.dropna(inplace = True)

documentDF.isna().sum()

document    0
dtype: int64

In [39]:
filename = '../datas/kr_stopword.txt'
url = "https://gist.githubusercontent.com/chulgil/d10b18575a73778da4bc83853385465c/raw/a1a451421097fa9a93179cb1f1f0dc392f1f9da9/stopwords.txt"
ret = urlretrieve(url, filename)

In [40]:
# from sklearn.feature_extraction.text import CountVectorizer

# # 불용어 제거
# vector = CountVectorizer(stop_words=['ㅋㅋ','OOO', 'ㅠㅠㅠㅠㅠㅠ','ㅜㅜㅜㅜ', '^^'])


In [41]:
## 토큰화
from konlpy.tag import Okt

In [42]:
# 형태소분석기 인스턴스 생성
okt = Okt()

result = data.document.str.cat(data['document'])
result

0                                                   굳 ㅋ굳 ㅋ
1                 GDNTOPCLASSINTHECLUBGDNTOPCLASSINTHECLUB
2        뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아뭐야 이 평점들...
3        지루하지는 않은데 완전 막장임... 돈주고 보기에는....지루하지는 않은데 완전 막...
4        3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하...
                               ...                        
49995    오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함오랜만에 ...
49996    의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO의지...
49997    그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다그림도 좋고 완성도도 ...
49998    절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네...
49999                                 마무리는 또 왜이래마무리는 또 왜이래
Name: document, Length: 49997, dtype: object

In [43]:
vocab={}
for idx in range(documentDF.shape[0]): #DataFrame.shape[0] =행의 갯수
    result = okt.morphs(documentDF.iloc[idx][0]) # 문자열만.
    #print result
    for word in result:
        if len(word)>=2: #길이가 2 이상만 내놔봐
            if vocab.get(word):
                vocab[word]+=1
            else:
                vocab[word] = 1
    

In [47]:
len(vocab)

61640

In [ ]:
sorted(vocab, key=lambda a:a[1])

In [ ]:
sorted(vocab, reverse=False)

In [63]:
# 불용어 제거하기
stopwords = set(open(filename).read().splitlines())
#stopwords
clean = {token: value for token,value in vocab.items() if token not in stopwords}

clean

{'평점': 2189,
 '나쁘진': 12,
 '않지': 10,
 '만점': 178,
 '더욱': 90,
 '아니잖아': 16,
 '지루하지는': 4,
 '않은데': 22,
 '완전': 799,
 '막장': 209,
 '임돈': 2,
 '주고': 104,
 '보기': 523,
 '에는': 342,
 '아니었어도': 6,
 '줬을텐데': 1,
 '로나': 20,
 '심기': 1,
 '불편하게하죠': 1,
 '음악': 377,
 '주가': 8,
 '최고': 2014,
 '영화': 15200,
 '진정한': 95,
 '쓰레기': 1095,
 '미국': 209,
 '애니': 283,
 '튀어나온듯': 1,
 '창의력': 2,
 '없는': 1088,
 '로봇': 36,
 '디자인': 19,
 '부터가': 28,
 '고개': 39,
 '젖게한': 1,
 '갈수록': 195,
 '개판': 59,
 '되가는': 1,
 '중국영화': 32,
 '유치하고내용': 1,
 '없음': 288,
 '잡다': 2,
 '끝남말도': 1,
 '안되는': 191,
 '무기': 36,
 '유치한': 51,
 '남무': 1,
 '그립다': 24,
 '동사서독': 2,
 '같은': 815,
 '이건': 578,
 '류작': 25,
 '이다': 1664,
 '이별': 33,
 '아픔': 40,
 '찾아오는': 3,
 '새로운': 67,
 '인연': 20,
 '기쁨': 10,
 '모든': 363,
 '사람': 1609,
 '그렇지는': 3,
 '않네': 13,
 '괜찮네요': 29,
 '오랜': 214,
 '포켓몬스터': 4,
 '잼밌': 2,
 '한국': 403,
 '독립영화': 73,
 '한계': 50,
 '그렇게': 212,
 '아버지': 124,
 '된다와': 1,
 '비교': 152,
 '청춘': 62,
 '아름답': 7,
 '다그': 155,
 '아름다': 43,
 '이성': 11,
 '흔들어놓는다': 1,
 '찰나': 4,
 '포착': 2,
 '섬세하고': 7,
 '아름다운': 249,


In [54]:
# def cleaning(review, okt, remove_stopwords=False, stop_words = []):

#     import re
#     import string
#     # 파라미터
#     #review : 전처리할 텍스트 목록
#     #okt = okt 객체를 많이 만들지 말고 미리 생성해서 받아놓는다.
#     #remo_stopwords = 불용어 제거 여부: 기본은 false.
#     #stop_words = 불용어 리스트: 기본은 빈 리스트, 사용자가 직접 입력해야 한다. 

#     #1. 한글 / 공백을 제외한 문자를 모두 제거하기
#     reviewtext = re.sub("[^가-힣ㄱ-ㅎ]ㅏ-ㅣ\\s]","", review)

#     # 2. okt 객체 사용, 형태소 단위로 나누기
#     word_review = okt.morphs(reviewtext, stem=True)

#     if remove_stopwords:

#         # 불용어 제거(아마도 선택적)
#         word_review = [token for token in word_review if not token in stop_words]

#     return word_review

In [62]:
# stop_words = set(open(filename).read().splitlines())
# okt = Okt()

# clean_review = []
# for review in documentDF['document']:
#     # 빈 데이터에서 멈추지 않도록, 문자열일 때만 진행한다
#     if type(review)==str:
#         clean_review.append(cleaning(review, okt, remove_stopwords=True, stop_words=stop_words))
#     else:
#         #스트링 아니면 빈 값 추가하기
#         clean_review.append([])


# clean_review

IndentationError: expected an indented block (1444663887.py, line 14)

In [65]:
VocabDF = pd.Series(clean).to_frame()

VocabDF.head()

,0
평점,2189
나쁘진,12
않지,10
만점,178
더욱,90


[3] 데이터셋용 단어 사전 만들기 <hr>
- 최종 사용할 단어 수
- 특별한 의미의 문자 추가 : '< UNK >', '< PAD >'

In [67]:
VOCAB_DICT = {0: '<UNK>', 1: '<PAD>'}

for idx in range(len(VocabDF)):
  print(VocabDF.index[idx])
  VOCAB_DICT[idx+2] = VocabDF.index[idx]

VOCAB_DICT

평점
나쁘진
않지
만점
더욱
아니잖아
지루하지는
않은데
완전
막장
임돈
주고
보기
에는
아니었어도
줬을텐데
로나
심기
불편하게하죠
음악
주가
최고
영화
진정한
쓰레기
미국
애니
튀어나온듯
창의력
없는
로봇
디자인
부터가
고개
젖게한
갈수록
개판
되가는
중국영화
유치하고내용
없음
잡다
끝남말도
안되는
무기
유치한
남무
그립다
동사서독
같은
이건
류작
이다
이별
아픔
찾아오는
새로운
인연
기쁨
모든
사람
그렇지는
않네
괜찮네요
오랜
포켓몬스터
잼밌
한국
독립영화
한계
그렇게
아버지
된다와
비교
청춘
아름답
다그
아름다
이성
흔들어놓는다
찰나
포착
섬세하고
아름다운
수채화
퀴어
보이는
반전
이었지만
흡인
사라지지
않았다
스토리
출연
기비
주얼
기본
안된
을해
찍고도
김문옥
감독
경력
인데
조무래기
들이내
평론
마인드
빠져있겠지
소위
가라
발연기
도저히
보겠다
진짜
이렇게
연기
할거라곤
상상
했네
나이스
재미
없는거우려
먹어
챔프
방송
몇번
했더라
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
금요일
나이트메어
시리즈
가장
많은
양산
해냈던
헬레
첫편
작가
상상력
돋보이는
작품
이며
갈고리
사지
찢는고어
지금
보더
라도
상당히
잔인하고
충격
나름
교훈
돋기는하지만
어쩔
수없이
받을수밖에
섹스
코미디
재밌게
였다
민주화
대의
억눌린
영혼
관음
욕구
분출
인상
천황
으로부터
받은
면죄부
긴박한
과정
루즈
하고
지저분하게
늘어놓았다
괜히
나올
어미
건드려서
긁어
부스
만들었다는
아래
보이
던데
영화제
대로
봤네
까지면서
새끼
나오려
했음
그냥
가면
한테
당했을것
놈막
나올게
뻔했으니
작살
낼수밖에
없었다
번은
봤네요
어쩜
만들었을까
실화
라서
아름답고
찡하네요
많이
울었어요
벌써
이란
흘렀네요
노답
이네
년대
어린이
보고
본받아라
마음
따뜻해지는
너무
재밌다
안보
후회
ㅠㅠ
점도
주기
싫어지는
영화배우
이라는
이나영
화판
봐으면
한다
알바생
무티
정말
표현
할수
액션
시나리오
왜그러지
막판
대동단결
나올것
없어요
아깝고
북괴
주적
ㅋㅋㅋ
난생처음
로그인
남기네요
빡쳐서
알바
속지
마세요
ㄱㅐ
낭비

{0: '<UNK>',
 1: '<PAD>',
 2: '평점',
 3: '나쁘진',
 4: '않지',
 5: '만점',
 6: '더욱',
 7: '아니잖아',
 8: '지루하지는',
 9: '않은데',
 10: '완전',
 11: '막장',
 12: '임돈',
 13: '주고',
 14: '보기',
 15: '에는',
 16: '아니었어도',
 17: '줬을텐데',
 18: '로나',
 19: '심기',
 20: '불편하게하죠',
 21: '음악',
 22: '주가',
 23: '최고',
 24: '영화',
 25: '진정한',
 26: '쓰레기',
 27: '미국',
 28: '애니',
 29: '튀어나온듯',
 30: '창의력',
 31: '없는',
 32: '로봇',
 33: '디자인',
 34: '부터가',
 35: '고개',
 36: '젖게한',
 37: '갈수록',
 38: '개판',
 39: '되가는',
 40: '중국영화',
 41: '유치하고내용',
 42: '없음',
 43: '잡다',
 44: '끝남말도',
 45: '안되는',
 46: '무기',
 47: '유치한',
 48: '남무',
 49: '그립다',
 50: '동사서독',
 51: '같은',
 52: '이건',
 53: '류작',
 54: '이다',
 55: '이별',
 56: '아픔',
 57: '찾아오는',
 58: '새로운',
 59: '인연',
 60: '기쁨',
 61: '모든',
 62: '사람',
 63: '그렇지는',
 64: '않네',
 65: '괜찮네요',
 66: '오랜',
 67: '포켓몬스터',
 68: '잼밌',
 69: '한국',
 70: '독립영화',
 71: '한계',
 72: '그렇게',
 73: '아버지',
 74: '된다와',
 75: '비교',
 76: '청춘',
 77: '아름답',
 78: '다그',
 79: '아름다',
 80: '이성',
 81: '흔들어놓는다',
 82: '찰나',
 83: '포착',
 84: '섬세하고',
 85: '

In [72]:
# 문장 수치화하기 : 인코딩
test = ["영화","최고", "생각"]

encode = []
sent = []
for tt in test:
    sent.append(0)
    for k, v in VOCAB_DICT.items():
        if v==tt:
            sent[-1]= k
            break
        
print(sent)

[24, 23, 621]


In [74]:
#decode
decoding = []
words = []
for tt in sent:
    for tt in sent:
        words.append(VOCAB_DICT.get(tt))

print(words)

['영화', '최고', '생각', '영화', '최고', '생각', '영화', '최고', '생각']


In [ ]:
VOCAB_DICT = {0:'<UNK>', 1:'<PAD>'}
for idx in range(10):
    VOCAB_DICT[idx+2] =VocabDF['Word'][idx]
VOCAB_DICT